In [49]:
from collections import defaultdict

In [59]:
import psycopg2
import snowflake.connector

# Connect to ElephantSQL
elephant_conn = psycopg2.connect(
    host='raja.db.elephantsql.com',
    database='dfpttbwb',
    user='dfpttbwb',
    password='S8fow-l1RjmBoNrey-qdjk0oh2FYZ06c'
)
elephant_cur = elephant_conn.cursor()

# Connect to Snowflake
snowflake_conn = snowflake.connector.connect(
    user='RUTHVIK',
    password='Ruthvik@161102',
    account='uxatedl-cp08110',
    warehouse='COMPUTE_WH',
    database='ORIGINAL_DB',
    schema='PUBLIC'
)
snowflake_cur = snowflake_conn.cursor()

# Query data from ElephantSQL table
def write_table_data(table_name):
    elephant_cur.execute('SELECT * FROM ' + table_name)
    rows = elephant_cur.fetchall()
    elephant_cur.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table_name}'")
    columns = elephant_cur.fetchall()
    snowflake_to_postgresql = {
        'integer': 'INTEGER',
        'character varying': 'VARCHAR',
        'date': 'DATE',
        'time without time zone': "TIMESTAMP WITHOUT TIME ZONE",
        'oid': "INTEGER",
        'bigint': "INTEGER",
        'text': "VARCHAR",
        'double precision': "INTEGER"
    }

    create_or_replace_table = 'CREATE OR REPLACE TABLE ' + table_name + ' ('
    for i in range(len(columns) - 1):
        print(columns[i])
        create_or_replace_table = create_or_replace_table + columns[i][0] + ' ' + snowflake_to_postgresql[columns[i][1]] + ', '

    create_or_replace_table = create_or_replace_table + columns[-1][0] + ' ' + snowflake_to_postgresql[columns[-1][1]] + ');'
    # Write data to Snowflake table
    print(create_or_replace_table, '\n\n')
    snowflake_cur.execute(create_or_replace_table)

    dtype_conversion = {
        'integer': "%s",
        'varchar': "%s",
        'character varying': '%s',
        'date': "%s",
        'timestamp without time zone': "%s",
        'time without time zone': "%s",
        'oid': "%d",
        'bigint': "%d",
        'text': "%s",
        'double precision': "%s"
    }

    insert_query = "INSERT INTO " + table_name + " VALUES ("
    for i in range(len(columns) - 1):
        insert_query = insert_query + dtype_conversion[ columns[i][1].lower() ] + ", "

    insert_query = insert_query + dtype_conversion[ columns[-1][1].lower() ] + ");"
    snowflake_cur.executemany(insert_query, rows)
    snowflake_conn.commit()

elephant_cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
table_names = elephant_cur.fetchall()
for table in range(min(len(table_names), 6)):
    print(table)
    write_table_data(table_names[table][0])

# Close connections

elephant_cur.close()
elephant_conn.close()
snowflake_cur.close()
snowflake_conn.close()

0
('user_id', 'integer')
('email', 'character varying')
('password', 'character varying')
('full_name', 'character varying')
('date_of_birth', 'date')
('gender', 'character varying')
('address', 'character varying')
('phone_number', 'character varying')
CREATE OR REPLACE TABLE users (user_id INTEGER, email VARCHAR, password VARCHAR, full_name VARCHAR, date_of_birth DATE, gender VARCHAR, address VARCHAR, phone_number VARCHAR, role VARCHAR); 


1
('user_id', 'integer')
('project_name', 'character varying')
('role', 'character varying')
CREATE OR REPLACE TABLE user_projects (user_id INTEGER, project_name VARCHAR, role VARCHAR, skill VARCHAR); 


2
('user_id', 'integer')
('url', 'character varying')
('certification_name', 'character varying')
('obtained_date', 'date')
('expiry_date', 'date')
('status', 'character varying')
('proficiency_level', 'integer')
('certification_id', 'character varying')
('issued_by', 'character varying')
CREATE OR REPLACE TABLE user_skills (user_id INTEGER, url V